In [1]:
from flask import Flask, render_template, request, jsonify
import streamlit as st
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from PIL import Image
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

stem = PorterStemmer()
lem = nltk.WordNetLemmatizer()
stop_word = nltk.corpus.stopwords.words("english")
 

app = Flask(__name__)

model = pickle.load(open("disastertweetsmodel", "rb"))
countvect = pickle.load(open("countvectorizer", "rb"))

def converter(x):
    
    # Tokenize the words
    process_1 = word_tokenize(x.lower())
    
    # Removing "#" from text 
    process_2 = [re.sub("#", "", i) for i in process_1]
    
    # Removing non-alphanumeric strings
    process_3 = [i for i in process_2 if i.isalpha()]
    
    # Removing stopwords
    process_4 = [i for i in process_3 if i not in stop_word]
        
    # Lemmatize words
    process_5 = [lem.lemmatize(i) for i in process_4]
    
    process_6 = [i for i in process_5 if i not in ["http", "https"]]
    
    return " ".join(process_6)




@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():

    tweet = converter(request.form.values())
    final_tweet = countvect.transform(np.array([tweet]))
    result = model.predict(final_tweet)
    
    if list(result)[0] == 0:
        return render_template("index.html", "This tweet is not related with disasters.")
    else:
        return render_template("index.html", "This tweet is  related with disasters.!!!")
    
@app.route("/results", methods=["POST"])  
def results():
    
    data = request.get_json(force=True)
    prediction = model.predict(countvect.transform(np.array([converter(list(data.values())[0])])))
    
    if list(prediction)[0] == 0:
        return jsonify("This tweet is not disaster related.")
    else:
        return jsonify("This tweet is related with disaster!!!!")
    
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rstsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rstsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rstsa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-01-18 12:40:42.375 INFO    werkzeug:  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2021-01-18 12:40:55.201 INFO    werkzeug: 127.0.0.1 - - [18/Jan/2021 12:40:55] "POST /results HTTP/1.1" 200 -
2021-01-18 12:42:16.124 INFO    werkzeug: 127.0.0.1 - - [18/Jan/2021 12:42:16] "POST /results HTTP/1.1" 200 -
